In [1]:
import systemlink.clients.nitestmonitor as testmon

In [9]:
import pandas as pd

In [2]:
results_api = testmon.ResultsApi()

In [5]:
results_filter = 'startedWithin <= "1.0:0:0"'
products_filter = ''

In [6]:
results_query = testmon.ResultsAdvancedQuery(
    results_filter, product_filter=products_filter)

results = []

response = await results_api.query_results_v2(post_body=results_query)
while response.continuation_token:
    results = results + response.results
    results_query.continuation_token = response.continuation_token
    response = await results_api.query_results_v2(post_body=results_query)

results_list = [result.to_dict() for result in results]

In [25]:
formatted_results = {
    'status': [result['status']['status_type'] for result in results_list],
    'started_at': [result['started_at'] for result in results_list],
    'location': [result['properties']['Location'] for result in results_list]
}

In [49]:
df = pd.DataFrame.from_dict(formatted_results)

In [50]:
df

,status,started_at,location
0,PASSED,2022-02-16 23:04:35.438661+00:00,EPL-AR2-Line 4
1,PASSED,2022-02-17 01:58:15.040950+00:00,EPL-AR2-Line 1
2,PASSED,2022-02-18 00:06:54.579457+00:00,EPL-AR1-Line 5
3,PASSED,2022-02-17 07:46:31.099331+00:00,EPL-AR1-Line 2
4,PASSED,2022-02-17 21:09:23.806703+00:00,EPL-AR2-Line 2
...,...,...,...
89131,PASSED,2022-02-17 04:08:04.101815+00:00,EPL-AR1-Line 6
89132,PASSED,2022-02-17 01:10:28.086104+00:00,EPL-AR2-Line 5
89133,PASSED,2022-02-17 12:26:04.005589+00:00,EPL-AR2-Line 3
89134,PASSED,2022-02-16 22:18:17.569882+00:00,EPL-AR2-Line 2


In [51]:
filt = df['location'].str.contains('EPL')

In [52]:
df = df.loc[filt]

In [53]:
df = df.groupby(['location', 'status']).count()

In [54]:
df

started_at
location       status                
EPL-AR1-Line 1 FAILED             199
               PASSED            5377
               RUNNING              3
               TERMINATED           4
EPL-AR1-Line 2 FAILED             187
               PASSED            6098
               RUNNING              5
EPL-AR1-Line 5 FAILED             504
               PASSED            8614
               RUNNING              6
               TERMINATED          13
EPL-AR1-Line 6 FAILED             371
               PASSED            7456
               RUNNING              1
               TERMINATED           4
EPL-AR2-Line 1 FAILED             582
               PASSED           11867
               RUNNING              4
               TERMINATED           2
EPL-AR2-Line 2 FAILED             285
               PASSED            7974
               RUNNING              4
               TERMINATED           1
EPL-AR2-Line 3 FAILED             433
               PASSED            7833
               RUNNING              3
               TERMINATED           2
EPL-AR2-Line 4 FAILED             313
               PASSED            8487
               RUNNING              7
EPL-AR2-Line 5 FAILED             511
               PASSED            8154
               RUNNING              4

In [58]:
df = df.unstack()

In [68]:
df

started_at                            
status             FAILED   PASSED RUNNING TERMINATED
location                                             
EPL-AR1-Line 1      199.0   5377.0     3.0        4.0
EPL-AR1-Line 2      187.0   6098.0     5.0        NaN
EPL-AR1-Line 5      504.0   8614.0     6.0       13.0
EPL-AR1-Line 6      371.0   7456.0     1.0        4.0
EPL-AR2-Line 1      582.0  11867.0     4.0        2.0
EPL-AR2-Line 2      285.0   7974.0     4.0        1.0
EPL-AR2-Line 3      433.0   7833.0     3.0        2.0
EPL-AR2-Line 4      313.0   8487.0     7.0        NaN
EPL-AR2-Line 5      511.0   8154.0     4.0        NaN

In [69]:
columns = ['failed', 'passed', 'running', 'terminated']

In [70]:
df.columns = columns

In [71]:
df

,failed,passed,running,terminated
location,,,,
EPL-AR1-Line 1,199.0,5377.0,3.0,4.0
EPL-AR1-Line 2,187.0,6098.0,5.0,NaN
EPL-AR1-Line 5,504.0,8614.0,6.0,13.0
EPL-AR1-Line 6,371.0,7456.0,1.0,4.0
EPL-AR2-Line 1,582.0,11867.0,4.0,2.0
EPL-AR2-Line 2,285.0,7974.0,4.0,1.0
EPL-AR2-Line 3,433.0,7833.0,3.0,2.0
EPL-AR2-Line 4,313.0,8487.0,7.0,NaN
EPL-AR2-Line 5,511.0,8154.0,4.0,NaN


In [85]:
df.fillna(0, inplace=True)

In [86]:
df['pass_rate'] = df['passed'] / (df['passed'] + df['failed'] + df['terminated']) * 100

In [87]:
df

,failed,passed,running,terminated,pass_rate
location,,,,,
EPL-AR1-Line 1,199.0,5377.0,3.0,4.0,96.362007
EPL-AR1-Line 2,187.0,6098.0,5.0,0.0,97.024662
EPL-AR1-Line 5,504.0,8614.0,6.0,13.0,94.337970
EPL-AR1-Line 6,371.0,7456.0,1.0,4.0,95.211340
EPL-AR2-Line 1,582.0,11867.0,4.0,2.0,95.309614
EPL-AR2-Line 2,285.0,7974.0,4.0,1.0,96.537530
EPL-AR2-Line 3,433.0,7833.0,3.0,2.0,94.738752
EPL-AR2-Line 4,313.0,8487.0,7.0,0.0,96.443182
EPL-AR2-Line 5,511.0,8154.0,4.0,0.0,94.102712
